# Path e librerie
## Prima di modificare avvertire gli altri!

## Install

In [1]:
#import drive dir
from google.colab import drive
drive_dir = 'drive' ; 
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install polyglot
!pip install PyICU
!pip install pycld2
!pip install morfessor

     |████████████████████████████████| 126 kB 5.1 MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52578 sha256=bada7bf87d317e8ac077d787237bf85c5f3259fa775cfb1e171bd85883079dd1
  Stored in directory: /root/.cache/pip/wheels/09/bc/67/75c9de8e9726460bc0b101ad225ad025cb8ce9e0759beb9d52
Successfully built polyglot
     |████████████████████████████████| 299 kB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for PyICU: filename=PyICU-2.8-cp37-cp37m-linux_x86_64.whl size=1376966 sha256=125dcf7a384bf552531ea459419b8903c0e2e625d0780e0880a41e1022f5a5cf
  Stored in directory: /root/.cache/pip/wheels/14/bd/45/aeddc643bd0637c14fa27bffaee5b411cdc323f8bec76ad15e
Successfully built PyICU
     |████████████████████████████████| 41.4 MB 38 kB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834241 sha256=bcdee6926cab93cc4748e

## Import

In [3]:
import tweepy
import numpy as np
from tweepy import Stream, OAuthHandler
from tqdm.notebook import tqdm
# from tqdm import tqdm !! per Jupyter from tqdm.notebook non funziona, runnare questo
import pandas as pd
pd.set_option('display.max_colwidth', None) # setta il num. di caratteri visibili per ogni cella della colonna
# es. 50 = primi 50 caratteri
# None = testo completo

import io
import pprint
import os
import glob
import warnings
warnings.filterwarnings('ignore')

from polyglot.detect import Detector
import icu

## Path per Colab

In [4]:
##########################
### ROOT
##########################

prj_root_dir = f'/content/{drive_dir}/MyDrive/nucleare/'
os.makedirs(prj_root_dir, exist_ok=True)

##########################
### SCRAPING: file dello scraping di ATP
##########################

scrapingATP_root_dir = prj_root_dir + 'scrapingATP/'
os.makedirs(scrapingATP_root_dir, exist_ok=True)

##########################
### ETL: 
##########################

etl_root_dir = prj_root_dir + 'ETL/'
os.makedirs(etl_root_dir, exist_ok=True)

##########################
### DWH
##########################

dw_root_dir = prj_root_dir + 'DWH/'
os.makedirs(dw_root_dir, exist_ok=True)

##########################
### SENTIMENT
##########################

sentiment_root_dir = prj_root_dir + 'SENTIMENT/'
os.makedirs(sentiment_root_dir, exist_ok=True)

##########################
### WORDEMBEDDING
##########################

embedding_root_dir = prj_root_dir + 'EMBEDDING/'
os.makedirs(embedding_root_dir, exist_ok=True)

##########################
### OUTPUT
##########################

output_root_dir = prj_root_dir + 'OUTPUT/'
os.makedirs(output_root_dir, exist_ok=True)

##########################
### STAGING ANTONELLO M.
##########################

st_man_root_dir = prj_root_dir + 'STAGING_MANENTI/'
os.makedirs(dw_root_dir, exist_ok=True)

### Path per Jupyter
Solo quando si usa Jupyter runnare, il percorso dovrebbe essere lo stesso per tutti/e

In [ ]:
##########################
### ROOT
##########################

drive_dir = f'G:\\My Drive'
prj_root_dir = f'{drive_dir}\\nucleare\\'

##########################
### SCRAPING
##########################

scrapingATP_root_dir = prj_root_dir + 'scrapingATP\\'

##########################
### ETL
##########################

etl_root_dir = prj_root_dir + 'ETL\\'

##########################
### DWH
##########################

dw_root_dir = prj_root_dir + 'DWH\\'

##########################
### SENTIMENT
##########################

sentiment_root_dir = prj_root_dir + 'SENTIMENT\\'

##########################
### WORDEMBEDDING
##########################

embedding_root_dir = prj_root_dir + 'EMBEDDING\\'

##########################
### OUTPUT
##########################

output_root_dir = prj_root_dir + 'OUTPUT\\'

##########################
### STAGING ANTONELLO M.
##########################

st_man_root_dir = prj_root_dir + 'STAGING_MANENTI\\'

# 1. Ingest
## Antonello

cartella di scraping: nulceare/scrapingATP

## Codici Twitter

In [ ]:
consumer_key = 'WYOB4KubrmdOM9ycdyGvBLkTa'# API Key
consumer_secret = 'AThpoW6fKDm6bwZYeO0C0AVM7jKVq0bVgU7w9VfMejrnkygBTK' # APY Secret Key
access_token = '1396896479525097474-2HgbagRtx3ByeO4zmkWSPaL1iOxEiH'
access_secret = 'IKvhdjwGMDky2dAN6OYdeMZhWUHYheqTmMEWdgcInXf4F'
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

# consumer_key = 'MQXChIn29dCky3pOQyW890Uxi'
# consumer_secret = 'u4n3UeFz58b4H6InWFhBDUPZsfJ8xdVS0wODdnfAjz6e6PUbCJ'
# access_token = '262247836-oWkm6AZIw4K0Fldnt8VBXwuL9IbtNCZwVj73ODkI'
# access_secret = 'GySGl7BGaYvmXUtCapu5pHwEDHD5tA3Qd4yCMIDPKm1Wx'
# auth = OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)
# api = tweepy.API(auth, wait_on_rate_limit=True)

# consumer_key = 'Vzk6O5t967f1n4iz9iQcsQwhf'
# consumer_secret = 'Q8sncdY1j85mwNitOqb5Spcg6U6uuZ0E2KNHHZEYF2J4DdCyVi'
# access_token = '164224789-mFmU5YhessCNvubNojgfL605tbU0vh0NOna3HdQU'
# access_secret = 'Ic5LNOH2QqxziSq4eTQdJvbXo3noTcQDUzCKXgFcdPzpC'
# auth = OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)
# api = tweepy.API(auth, wait_on_rate_limit=True)

# consumer_key = 'A4GRiWdUDgiGPckQSykzA'
# consumer_secret = 'nSyjpFDTr3eEmd8tFeBM1lwVrTNPLaHrcx0DicTEEuw'
# access_token = '1678747572-lbkg1jxpe5HypoTRSnRMeiVIe0RBMxhdjLJ0GJ6'
# access_secret = '1qVP0pHXV9HDgKhHDIy2HHqqnHFBm3sI5AR2ZDwZxlM'
# auth = OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)
# api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
utente = 'antonello' #mettere il proprio nome
lang = 'all' #mettere la lingua
prog = '20' #mettere il progressivo

## Scraping dello storico

In [ ]:
#mettere la query
query_00 = ('NittoATPFinals' or 'TennisTakesOverTorino' or '#ATPFinals' or 'ATPFinals' or 'ATP Finals' or 'Norrie' or '#Norrie' or '@cam_norrie' or 'Norrie' or '#Norrie' or '@cam_norrie' or 'Ruud' or '#Ruud' or '@CasperRuud98' or 'Rublev' or '#Rublev' or '@AndreyRublev97' or 'Tsitsipas' or '#Tsitsipas' or '@steftsitsipas' or 'Djokovic' or '#Djokovic' or '@DjokerNole' or 'Sinner' or '#Sinner' or '@janniksin' or 'Medvedev' or '#Medvedev' or '@DaniilMedwed' or 'Zverev' or '#Zverev' or '@AlexZverev' 'Berrettini' or '#Berrettini' or '@MattBerrettini' or 'Hurkacz' or '#Hurkacz' or '@HubertHurkacz')
query_01 = ('@DaniilMedwed' or 'Medvedev' or '#Medvedev' or '@AlexZverev' or 'Zverev' or '#Zverev' or '#NittoATPFinals' or 'NittoATPFinals' or 'TennisTakesOverTorino' or '#ATPFinals' or 'ATPFinals' or 'ATP Finals')

In [ ]:
searched_tweets = []
last_id = -1
max_tweets = 500_000
pbar = tqdm(total=max_tweets)

while len(searched_tweets) < max_tweets:
    count = max_tweets - len(searched_tweets)
    try:
        # api.search è un metodo che su colab funziona anche se non è documentato in Tweepy. Su Jupyter non funziona, sostituire con search_tweets
        new_tweets = api.search(q=query_01, count=1000, max_id=str(last_id - 1), tweet_mode="extended") # tweet_mode="extended" importante, altrimenti il testo del tweet viene troncato
        # If I've reached the end of the search then I'm done
        # immediately the cycle while
        if not new_tweets:
            break
        # added the data found to the list
        searched_tweets.extend(new_tweets)
        # tqdm
        pbar.update(len(new_tweets))
        # retrieve the last id found
        last_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        print(e)
        break

pbar.close()

  0%|          | 0/500000 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
print('Totale Tweet')
len(searched_tweets)

Totale Tweet


12199

In [ ]:
dict_ = {'id': [], 'user': [], 'date': [], 'text': [], 
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}

for status_j in searched_tweets[0:max_tweets]:
    status = status_j._json

    if 'retweeted_status' in status:
      if 'extended_tweet' in status['retweeted_status']:
        text = status['retweeted_status']['extended_tweet']['full_text']
        dict_['text'].append(text)
      else:
        text = status['retweeted_status']['full_text']
        dict_['text'].append(text)
    else:
      if 'extended_tweet' in status:
        text = status['extended_tweet']['full_text']
        dict_['text'].append(text)
      else:
        text = status['full_text']
        dict_['text'].append(text)
    
    dict_['id'].append(status['id'])
    dict_['user'].append(status['user']['screen_name'])
    dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
    dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
    dict_['location'].append(status['user']['location'])
    dict_['followers_count'].append(status['user']['followers_count'])
    dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
    dict_['date'].append(status['created_at'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['retweet_count'].append(status['retweet_count'])
    dict_['retweet'].append(status['retweeted'])

df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index("id")
df.sort_values(by=['date'], inplace=True, ascending=True)
df.head(10)

,user,date,text,favorite_count,hashtags,location,retweet,retweet_count,followers_count,in_reply_to_status_id,user_mentions
id,,,,,,,,,,,
1460034895997243394,ValdoOlga,Mon Nov 15 00:00:19 +0000 2021,La “firma” sulla telecamera di Zverev.\nE le p...,0,[],,False,46,120,NaN,[toMMilanello]
1460035000661860360,Zverev_fans,Mon Nov 15 00:00:44 +0000 2021,☹️☹️☹️\n\nAll of us after Berrettini is forced...,0,[NittoATPFinals],,False,173,417,NaN,[TennisTV]
1460035118458888198,RaiSport,Mon Nov 15 00:01:13 +0000 2021,#Berrettini costretto ad abbandonare il match ...,52,"[Berrettini, Zverev, LaDS]",,False,6,247865,NaN,[]
1460035123047723010,mantas____,Mon Nov 15 00:01:14 +0000 2021,Not how any of us wanted this one to end 😢\n\n...,0,[NittoATPFinals],,False,304,6,NaN,[TennisTV]
1460035259551162373,Cucciolina96251,Mon Nov 15 00:01:46 +0000 2021,L’infortunio di Matteo #Berrettini mi distrugg...,0,[Berrettini],,False,23,122,NaN,[Alenize82]
1460035274663211012,Zverev_fans,Mon Nov 15 00:01:50 +0000 2021,'I'm sure it's not the last time we're gonna s...,0,[],,False,41,417,NaN,"[TennisTV, AlexZverev]"
1460035452744970244,malaymail,Mon Nov 15 00:02:32 +0000 2021,"Medvedev fights back to sink Hurkacz, Berretti...",1,[],Malaysia,False,0,267422,NaN,[]
1460035535997652999,nerdswire,Mon Nov 15 00:02:52 +0000 2021,Zverev gewinnt Auftakt bei ATP Finals https://...,0,[],"NRW, Deutschland",False,0,1457,NaN,[]
1460035698422075392,Martii063,Mon Nov 15 00:03:31 +0000 2021,Alexander Zverev vainqueur par abandon de Matt...,0,[],France,False,0,598,NaN,[]


In [ ]:
df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index("id")
df.sort_values(by=['date'], inplace=True, ascending=True)
df.head(10)

,user,date,text,favorite_count,hashtags,location,retweet,retweet_count,followers_count,in_reply_to_status_id,user_mentions
id,,,,,,,,,,,
1460034895997243394,ValdoOlga,Mon Nov 15 00:00:19 +0000 2021,La “firma” sulla telecamera di Zverev.\nE le p...,0,[],,False,46,120,NaN,[toMMilanello]
1460035000661860360,Zverev_fans,Mon Nov 15 00:00:44 +0000 2021,☹️☹️☹️\n\nAll of us after Berrettini is forced...,0,[NittoATPFinals],,False,173,417,NaN,[TennisTV]
1460035118458888198,RaiSport,Mon Nov 15 00:01:13 +0000 2021,#Berrettini costretto ad abbandonare il match ...,52,"[Berrettini, Zverev, LaDS]",,False,6,247865,NaN,[]
1460035123047723010,mantas____,Mon Nov 15 00:01:14 +0000 2021,Not how any of us wanted this one to end 😢\n\n...,0,[NittoATPFinals],,False,304,6,NaN,[TennisTV]
1460035259551162373,Cucciolina96251,Mon Nov 15 00:01:46 +0000 2021,L’infortunio di Matteo #Berrettini mi distrugg...,0,[Berrettini],,False,23,122,NaN,[Alenize82]
1460035274663211012,Zverev_fans,Mon Nov 15 00:01:50 +0000 2021,'I'm sure it's not the last time we're gonna s...,0,[],,False,41,417,NaN,"[TennisTV, AlexZverev]"
1460035452744970244,malaymail,Mon Nov 15 00:02:32 +0000 2021,"Medvedev fights back to sink Hurkacz, Berretti...",1,[],Malaysia,False,0,267422,NaN,[]
1460035535997652999,nerdswire,Mon Nov 15 00:02:52 +0000 2021,Zverev gewinnt Auftakt bei ATP Finals https://...,0,[],"NRW, Deutschland",False,0,1457,NaN,[]
1460035698422075392,Martii063,Mon Nov 15 00:03:31 +0000 2021,Alexander Zverev vainqueur par abandon de Matt...,0,[],France,False,0,598,NaN,[]


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12199 entries, 1460034895997243394 to 1461120978436734981
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   user                   12199 non-null  object 
 1   date                   12199 non-null  object 
 2   text                   12199 non-null  object 
 3   favorite_count         12199 non-null  int64  
 4   hashtags               12199 non-null  object 
 5   location               12199 non-null  object 
 6   retweet                12199 non-null  bool   
 7   retweet_count          12199 non-null  int64  
 8   followers_count        12199 non-null  int64  
 9   in_reply_to_status_id  1645 non-null   float64
 10  user_mentions          12199 non-null  object 
dtypes: bool(1), float64(1), int64(3), object(6)
memory usage: 1.0+ MB


In [ ]:
print(df.shape)

(12199, 11)


In [ ]:
df.to_csv(f'{scrapingATP_root_dir}twitter_history_{utente}_{prog}_{lang}_{len(searched_tweets)}.csv')

## Scraping dello stream

In [ ]:
class MyStreamListener(tweepy.StreamListener):
    data = [] #qui ci metto i tweet che scarico man mano
    num = 0 #questo è un contatore settato a 0

    def store(self):
      dict_ = {'id': [], 'user': [], 'date': [], 'text': [],
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}
      
      for status_j in self.data:
          status = status_j._json

          if 'retweeted_status' in status:
            if 'extended_tweet' in status['retweeted_status']:
              text = status['retweeted_status']['extended_tweet']['full_text']
              dict_['text'].append(text)
            else:
              text = status['retweeted_status']['full_text']
              dict_['text'].append(text)
          else:
            if 'extended_tweet' in status:
              text = status['extended_tweet']['full_text']
              dict_['text'].append(text)
            else:
              text = status['full_text']
              dict_['text'].append(text)

          dict_['id'].append(status['id'])
          dict_['user'].append(status['user']['screen_name'])
          dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
          dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
          dict_['location'].append(status['user']['location'])
          dict_['followers_count'].append(status['user']['followers_count'])
          dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
          dict_['date'].append(status['created_at'])
          dict_['full_text'].append(status['full_text'])
          dict_['favorite_count'].append(status['favorite_count'])
          dict_['retweet_count'].append(status['retweet_count'])
          dict_['retweet'].append(status['retweeted'])

      df = pd.DataFrame.from_dict(dict_, orient="columns")
      df = df.set_index("id")
      df.sort_values(by='favorite_count', inplace=True, ascending=False)
      df.to_csv(f'{scrapingATP_root_dir}_twitter_stream_{utente}_{prog}_{lang}_' + str(self.num) + '.csv') #salvo in csv
      self.num = self.num + 1
      self.data = []


    def on_status(self, status):
        self.data.append(status)
        # to pandas
        # store su google drive
        if len(self.data) > 5000:
          self.store()

time: 37.9 ms (started: 2021-11-15 22:17:42 +00:00)


In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener, tweet_mode="extended")

time: 2.06 ms (started: 2021-11-15 22:17:45 +00:00)


In [ ]:
query_00 = ('NittoATPFinals' or 'TennisTakesOverTorino' or '#ATPFinals' or 'ATPFinals' or 'ATP Finals' or 'Norrie' or '#Norrie' or '@cam_norrie' or 'Norrie' or '#Norrie' or '@cam_norrie' or 'Ruud' or '#Ruud' or '@CasperRuud98' or 'Rublev' or '#Rublev' or '@AndreyRublev97' or 'Tsitsipas' or '#Tsitsipas' or '@steftsitsipas' or 'Djokovic' or '#Djokovic' or '@DjokerNole' or 'Sinner' or '#Sinner' or '@janniksin' or 'Medvedev' or '#Medvedev' or '@DaniilMedwed' or 'Zverev' or '#Zverev' or '@AlexZverev' 'Berrettini' or '#Berrettini' or '@MattBerrettini' or 'Hurkacz' or '#Hurkacz' or '@HubertHurkacz')
query_01 = ['@DaniilMedwed' or 'Medvedev' or '#Medvedev' or '@AlexZverev' or 'Zverev' or '#Zverev' or '#NittoATPFinals' or 'NittoATPFinals' or 'TennisTakesOverTorino' or '#ATPFinals' or 'ATPFinals' or 'ATP Finals']
myStream.filter(track=query_01)

KeyboardInterrupt: ignored

time: 4.37 s (started: 2021-11-15 22:17:51 +00:00)


In [ ]:
len(myStreamListener.data)

0

time: 4.37 ms (started: 2021-11-15 22:18:02 +00:00)


In [ ]:
dict_ = {'id': [], 'user': [], 'date': [], 'text': [], 
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}
#
for status_j in myStreamListener.data:
    status = status_j._json

    if 'retweeted_status' in status:
      if 'extended_tweet' in status['retweeted_status']:
        text = status['retweeted_status']['extended_tweet']['full_text']
        dict_['text'].append(text)
      else:
        text = status['retweeted_status']['text']
        dict_['text'].append(text)
    else:
      if 'extended_tweet' in status:
        text = status['extended_tweet']['full_text']
        dict_['text'].append(text)
      else:
        text = status['text']
        dict_['text'].append(text)

    dict_['id'].append(status['id'])
    dict_['user'].append(status['user']['screen_name'])
    dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
    dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
    dict_['location'].append(status['user']['location'])
    dict_['followers_count'].append(status['user']['followers_count'])
    dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
    dict_['date'].append(status['created_at'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['retweet_count'].append(status['retweet_count'])
    dict_['retweet'].append(status['retweeted'])

In [ ]:
df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index("id")
df.sort_values(by='favorite_count', inplace=True, ascending=False)
df.head(10)

,user,date,text,favorite_count,hashtags,location,retweet,retweet_count,followers_count,in_reply_to_status_id,user_mentions
id,,,,,,,,,,,
1460003463719440388,TennisTV,Sun Nov 14 21:55:25 +0000 2021,Not how any of us wanted this one to end 😢\n\n...,2696,[NittoATPFinals],Watch ATP Tour tournaments 👉,False,304,420515,NaN,[]
1460635739532582912,atptour,Tue Nov 16 15:47:52 +0000 2021,Relentless 💪 \n\n🇷🇺 @DaniilMedwed battles past...,1832,[NittoATPFinals],Follow the ATP Tour ➡,False,188,1690836,NaN,[DaniilMedwed]
1460003037032824836,atptour,Sun Nov 14 21:53:44 +0000 2021,💔\n\nMatteo Berrettini is forced to retire aga...,1824,[NittoATPFinals],Follow the ATP Tour ➡,False,146,1690836,NaN,[]
1460005336006463488,ItaliaTeam_it,Sun Nov 14 22:02:52 +0000 2021,A causa di un infortunio Matteo Berrettini è c...,1695,"[NittoATPFinals, ItaliaTeam]",Italia,False,158,103925,NaN,"[MattBerrettini, federtennis]"
1460006164935200772,FBiasin,Sun Nov 14 22:06:09 +0000 2021,Gran bel gesto di #Zverev.\n\n#Berrettini http...,1554,"[Zverev, Berrettini]",,False,53,141643,NaN,[]
1460002788948287489,TennisTV,Sun Nov 14 21:52:45 +0000 2021,Devastating 😔\n\nMatteo Berrettini pulls up wi...,1315,[NittoATPFinals],Watch ATP Tour tournaments 👉,False,135,420515,NaN,[]
1460003575149604869,romeoagresti,Sun Nov 14 21:55:52 +0000 2021,Che bello il gesto di #Zverev —&gt; ☹️,869,[Zverev],Torino,False,38,134357,NaN,[]
1460621045644234757,atptour,Tue Nov 16 14:49:28 +0000 2021,Zverev's not done yet 😳 We're going 3!\n\nMedv...,697,[NittoATPFinals],Follow the ATP Tour ➡,False,57,1690836,NaN,[]
1461337322276855811,josemorgado,Thu Nov 18 14:15:42 +0000 2021,Medvedev-Sinner will be a dead rubber.\n\nAbso...,606,[],"Lisbon, Portugal",False,34,83821,NaN,[]


In [ ]:
df.to_csv(f'{scrapingATP_root_dir}twitter_stream_{utente}_{prog}_{lang}_{len(myStreamListener.data)}.csv') #scarico l'ultimo stream

In [ ]:
#unisco tutti i csv

path = scrapingATP_root_dir

all_files = glob.glob(os.path.join(path, "twitter_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged = pd.concat(df_from_each_file, ignore_index=True)
df_merged.drop_duplicates(subset=['id'], inplace=True, ignore_index=False)
df_merged.to_csv(f'{etl_root_dir}twitter_merged_from_scraping.csv', index=False, sep=',', encoding='utf-8')

In [ ]:
df_merge = pd.read_csv(f'{etl_root_dir}twitter_merged_from_scraping.csv')
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33480 entries, 0 to 33479
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     33480 non-null  int64  
 1   user                   33480 non-null  object 
 2   date                   33480 non-null  object 
 3   text                   33480 non-null  object 
 4   favorite_count         33480 non-null  int64  
 5   hashtags               33480 non-null  object 
 6   location               19827 non-null  object 
 7   retweet                33480 non-null  bool   
 8   retweet_count          33480 non-null  int64  
 9   followers_count        33480 non-null  int64  
 10  in_reply_to_status_id  2999 non-null   float64
 11  user_mentions          33480 non-null  object 
dtypes: bool(1), float64(1), int64(4), object(6)
memory usage: 2.8+ MB


In [ ]:
df_merge.tail()

,id,user,date,text,favorite_count,hashtags,location,retweet,retweet_count,followers_count,in_reply_to_status_id,user_mentions
58124,1461119368969039882,Recupxrateur,Wed Nov 17 23:49:38 +0000 2021,🎯 | #ATPFinals \n\nZverev 2-0 | 1.67 | 1%\nMed...,0,"['ATPFinals', 'TeamParieur']",NaN,False,0,4,NaN,[]
58125,1461120133384994820,raihannb,Wed Nov 17 23:52:40 +0000 2021,Vigrin ATP: afraid to address Zverev's case\nC...,0,[],Australia,False,76,997,NaN,['_silver912']
58126,1461120695748997123,Marcelo21142,Wed Nov 17 23:54:54 +0000 2021,#NittoATPFinals Day 5 Media Notes: https://t.c...,0,['NittoATPFinals'],RJ,False,18,153,NaN,"['ATPMediaInfo', 'DaniilMedwed', 'janniksin']"
58127,1461120927844999172,MnardShady,Wed Nov 17 23:55:50 +0000 2021,"@SG_Lockwood Bah ecoute, chacun son delire. De...",0,[],"Paris, France",False,0,71,1.461121e+18,['SG_Lockwood']
58128,1461120978436734981,PokornyRoland,Wed Nov 17 23:56:02 +0000 2021,@TheBorisBecker \n\nHi Boris.\n\nwill you comm...,0,[],NaN,False,0,15,NaN,['TheBorisBecker']


# 2. ETL
**NATALIA**



1.   rimuovere #
2.   rimuovere id doppi
1.   rimuovere http
2.   rimuovere twitter con testo vuoto
1.   rimuovere @



file input: nucleare/ETL/twitter_merged_from_scraping.csv

cartella output: nucleare/DWH/

In [5]:
df = pd.read_csv(etl_root_dir+'twitter_merged_from_scraping.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84912 entries, 0 to 84911
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     84912 non-null  int64  
 1   user                   84912 non-null  object 
 2   date                   84912 non-null  object 
 3   text                   84912 non-null  object 
 4   favorite_count         84912 non-null  int64  
 5   hashtags               84912 non-null  object 
 6   location               50798 non-null  object 
 7   retweet                84912 non-null  bool   
 8   retweet_count          84912 non-null  int64  
 9   followers_count        84912 non-null  int64  
 10  in_reply_to_status_id  11407 non-null  float64
 11  user_mentions          84912 non-null  object 
dtypes: bool(1), float64(1), int64(4), object(6)
memory usage: 7.2+ MB


In [7]:
df.shape

(84912, 12)

In [8]:
df.isna().sum()

id                           0
user                         0
date                         0
text                         0
favorite_count               0
hashtags                     0
location                 34114
retweet                      0
retweet_count                0
followers_count              0
in_reply_to_status_id    73505
user_mentions                0
dtype: int64

In [9]:
df_clean=df

In [10]:
df_clean = df_clean.drop_duplicates(subset='id', keep="first")

In [11]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84912 entries, 0 to 84911
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     84912 non-null  int64  
 1   user                   84912 non-null  object 
 2   date                   84912 non-null  object 
 3   text                   84912 non-null  object 
 4   favorite_count         84912 non-null  int64  
 5   hashtags               84912 non-null  object 
 6   location               50798 non-null  object 
 7   retweet                84912 non-null  bool   
 8   retweet_count          84912 non-null  int64  
 9   followers_count        84912 non-null  int64  
 10  in_reply_to_status_id  11407 non-null  float64
 11  user_mentions          84912 non-null  object 
dtypes: bool(1), float64(1), int64(4), object(6)
memory usage: 7.9+ MB


In [12]:
df_clean['text_nohash']=df_clean['text']

In [13]:
# rimuovo gli hashtag
df_clean['text_nohash']=df_clean['text_nohash'].replace('#\w+ ?','', regex=True)
#stringwithouthash = re.sub(r'#\w+ ?', '', tweet['text'])

In [14]:
# rimuovo le mention
df_clean['text_nohash_men_']=df_clean['text_nohash'].replace('@\w+ ?','', regex=True)
#stringwithouthash = re.sub(r'@\w+ ?', '', tweet['text'])

In [15]:
# rimuovo i link
df_clean['text_nohash_men_http']=df_clean['text_nohash_men_'].replace('http\S+','', regex=True)
#stringwithoutlink = re.sub(r'http\S+', '', tweet['text'])

In [16]:
#df_clean['text_cleaned'] = df_clean['text_cleaned'].str.replace('\n','', regex=True)

In [17]:
df_clean['text_nohash_men_http_nl'] = df_clean['text_nohash_men_http'].apply(lambda x: x.replace('\n','') + '\n')

In [18]:
#df_clean['text_nohash_men_http_nl'] = df_clean['text_nohash_men_http'].replace(r'\\n',' ', regex=True)

In [19]:
# rimuovo i doppi spazi
df_clean['text_nohash_men_http_nl_2spazi'] = df_clean['text_nohash_men_http_nl'].str.replace('\s+',' ', regex=True)

In [20]:
df_clean['text_nohash_men_http_nl_2spazi']

0                                                             Rublev’s revenge 😤reverses the result from the 2020 and gets the win over Tsitsipas! 
1                                                      Red-hot start 🔥powers past 2019 champion Tsitsipas 6-4 6-4 to get his first win of the 2021 
2                                                                                                                                 Alexander Zverev 
3                                                                                                                            When the 🐺 met the 🦁🤝 
4                                                                                                                                           💙🤍💙🤍🇬🇷 
                                                                            ...                                                                    
84907                                                                                            From London to 

In [21]:
#rimuovo i numeri
df_clean['text_nohash_men_http_nl_2spazi_num'] = df_clean['text_nohash_men_http_nl_2spazi'].str.replace('\d+', '')

In [22]:
# rimuovo le emoticon e la punteggiatura
df_clean['text_nohash_men_http_nl_2spazi_num_simboli']=df_clean['text_nohash_men_http_nl_2spazi_num'].str.replace('[^\w\s]',' ', regex=True)

In [23]:
df_clean.shape

(84912, 19)

In [24]:
df_clean['text_nohash_men_http_nl_2spazi_num_simboli'].head()

0    Rublev s revenge  reverses the result from the  and gets the win over Tsitsipas  
1     Red hot start  powers past  champion Tsitsipas     to get his first win of the  
2                                                                    Alexander Zverev 
3                                                               When the   met the    
4                                                                                     
Name: text_nohash_men_http_nl_2spazi_num_simboli, dtype: object

In [25]:
df_clean['text_nohash_men_http_nl_2spazi_num_simboli'] = df_clean['text_nohash_men_http_nl_2spazi_num_simboli'].str.replace('\s+',' ', regex=True)

In [26]:
df_clean['text_nohash_men_http_nl_2spazi_num_simboli'].head()

0    Rublev s revenge reverses the result from the and gets the win over Tsitsipas 
1         Red hot start powers past champion Tsitsipas to get his first win of the 
2                                                                 Alexander Zverev 
3                                                                 When the met the 
4                                                                                  
Name: text_nohash_men_http_nl_2spazi_num_simboli, dtype: object

In [27]:
df_clean.to_csv(dw_root_dir+'twitter_text_cleaned.csv', index=False, sep=',',encoding='utf-8')
#df_clean.to_csv(dw_root_dir+'twitter_text_cleaned.csv')
df = pd.read_csv(dw_root_dir+'twitter_text_cleaned.csv', sep=',')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84912 entries, 0 to 84911
Data columns (total 19 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   id                                          84912 non-null  int64  
 1   user                                        84912 non-null  object 
 2   date                                        84912 non-null  object 
 3   text                                        84912 non-null  object 
 4   favorite_count                              84912 non-null  int64  
 5   hashtags                                    84912 non-null  object 
 6   location                                    50798 non-null  object 
 7   retweet                                     84912 non-null  bool   
 8   retweet_count                               84912 non-null  int64  
 9   followers_count                             84912 non-null  int64  
 10  in_reply_t

# 3. Language detection
ANTONELLO M.

In [28]:
df = pd.read_csv(f'{dw_root_dir}twitter_text_cleaned.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84912 entries, 0 to 84911
Data columns (total 19 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   id                                          84912 non-null  int64  
 1   user                                        84912 non-null  object 
 2   date                                        84912 non-null  object 
 3   text                                        84912 non-null  object 
 4   favorite_count                              84912 non-null  int64  
 5   hashtags                                    84912 non-null  object 
 6   location                                    50798 non-null  object 
 7   retweet                                     84912 non-null  bool   
 8   retweet_count                               84912 non-null  int64  
 9   followers_count                             84912 non-null  int64  
 10  in_reply_t

In [29]:
df.sample(n=5, random_state=13)

,id,user,date,text,favorite_count,hashtags,location,retweet,retweet_count,followers_count,in_reply_to_status_id,user_mentions,text_nohash,text_nohash_men_,text_nohash_men_http,text_nohash_men_http_nl,text_nohash_men_http_nl_2spazi,text_nohash_men_http_nl_2spazi_num,text_nohash_men_http_nl_2spazi_num_simboli
70772,1462148394663751682,laurapetrarca2,Sat Nov 20 19:58:37 +0000 2021,These are the matches you want to play in.\n\n🇷🇸 Djokovic v. Zverev 🇩🇪\n\nIt's a blockbuster in Turin. #NittoATPFinals https://t.co/kCOMHO31Vk,0,['NittoATPFinals'],NaN,False,53,770,NaN,['atptour'],These are the matches you want to play in.\n\n🇷🇸 Djokovic v. Zverev 🇩🇪\n\nIt's a blockbuster in Turin. https://t.co/kCOMHO31Vk,These are the matches you want to play in.\n\n🇷🇸 Djokovic v. Zverev 🇩🇪\n\nIt's a blockbuster in Turin. https://t.co/kCOMHO31Vk,These are the matches you want to play in.\n\n🇷🇸 Djokovic v. Zverev 🇩🇪\n\nIt's a blockbuster in Turin.,These are the matches you want to play in.🇷🇸 Djokovic v. Zverev 🇩🇪It's a blockbuster in Turin. \n,These are the matches you want to play in.🇷🇸 Djokovic v. Zverev 🇩🇪It's a blockbuster in Turin.,These are the matches you want to play in.🇷🇸 Djokovic v. Zverev 🇩🇪It's a blockbuster in Turin.,These are the matches you want to play in Djokovic v Zverev It s a blockbuster in Turin
9482,1460729576611454978,shapo_fan,Tue Nov 16 22:00:44 +0000 2021,From one to the other 👊\n\nThe only two Italians to win a singles match at the ATP Finals: Jannik Sinner &amp; Matteo Berrettini\n\n@janniksin @MattBerrettini #NittoATPFinals https://t.co/7mFVhgiY4u,0,[],Japan,False,88,407,NaN,['TennisTV'],From one to the other 👊\n\nThe only two Italians to win a singles match at the ATP Finals: Jannik Sinner &amp; Matteo Berrettini\n\n@janniksin @MattBerrettini https://t.co/7mFVhgiY4u,From one to the other 👊\n\nThe only two Italians to win a singles match at the ATP Finals: Jannik Sinner &amp; Matteo Berrettini\n\nhttps://t.co/7mFVhgiY4u,From one to the other 👊\n\nThe only two Italians to win a singles match at the ATP Finals: Jannik Sinner &amp; Matteo Berrettini\n\n,From one to the other 👊The only two Italians to win a singles match at the ATP Finals: Jannik Sinner &amp; Matteo Berrettini\n,From one to the other 👊The only two Italians to win a singles match at the ATP Finals: Jannik Sinner &amp; Matteo Berrettini,From one to the other 👊The only two Italians to win a singles match at the ATP Finals: Jannik Sinner &amp; Matteo Berrettini,From one to the other The only two Italians to win a singles match at the ATP Finals Jannik Sinner amp Matteo Berrettini
22816,1460216257542774790,NellySkri22,Mon Nov 15 12:00:59 +0000 2021,Que c'est cruel pour Berrettini\n\nLe héros local doit abandonner en début de 1er set sur une blessure aux abdos alors qu'il faisait presque jeu égal avec Alexander Zverev (7-6 1-0 ab) #ATPFinals https://t.co/xrY3bYMxj0,0,[],"Amsterdam, Pays-Bas",False,16,43,NaN,['WeAreTennisFR'],Que c'est cruel pour Berrettini\n\nLe héros local doit abandonner en début de 1er set sur une blessure aux abdos alors qu'il faisait presque jeu égal avec Alexander Zverev (7-6 1-0 ab) https://t.co/xrY3bYMxj0,Que c'est cruel pour Berrettini\n\nLe héros local doit abandonner en début de 1er set sur une blessure aux abdos alors qu'il faisait presque jeu égal avec Alexander Zverev (7-6 1-0 ab) https://t.co/xrY3bYMxj0,Que c'est cruel pour Berrettini\n\nLe héros local doit abandonner en début de 1er set sur une blessure aux abdos alors qu'il faisait presque jeu égal avec Alexander Zverev (7-6 1-0 ab),Que c'est cruel pour BerrettiniLe héros local doit abandonner en début de 1er set sur une blessure aux abdos alors qu'il faisait presque jeu égal avec Alexander Zverev (7-6 1-0 ab) \n,Que c'est cruel pour BerrettiniLe héros local doit abandonner en début de 1er set sur une blessure aux abdos alors qu'il faisait presque jeu égal avec Alexander Zverev (7-6 1-0 ab),Que c'est cruel pour BerrettiniLe héros local doit abandonner en début de er set sur une blessu

In [30]:
# testing polyglot
df_test = df[['text_nohash_men_http_nl_2spazi_num_simboli', 'text']]
df_test = df_test.sample(n=3, random_state=13)
df_test

,text_nohash_men_http_nl_2spazi_num_simboli,text
70772,These are the matches you want to play in Djokovic v Zverev It s a blockbuster in Turin,These are the matches you want to play in.\n\n🇷🇸 Djokovic v. Zverev 🇩🇪\n\nIt's a blockbuster in Turin. #NittoATPFinals https://t.co/kCOMHO31Vk
9482,From one to the other The only two Italians to win a singles match at the ATP Finals Jannik Sinner amp Matteo Berrettini,From one to the other 👊\n\nThe only two Italians to win a singles match at the ATP Finals: Jannik Sinner &amp; Matteo Berrettini\n\n@janniksin @MattBerrettini #NittoATPFinals https://t.co/7mFVhgiY4u
22816,Que c est cruel pour BerrettiniLe héros local doit abandonner en début de er set sur une blessure aux abdos alors qu il faisait presque jeu égal avec Alexander Zverev ab,Que c'est cruel pour Berrettini\n\nLe héros local doit abandonner en début de 1er set sur une blessure aux abdos alors qu'il faisait presque jeu égal avec Alexander Zverev (7-6 1-0 ab) #ATPFinals https://t.co/xrY3bYMxj0


In [31]:
df_test['poly_obj'] = df_test.text_nohash_men_http_nl_2spazi_num_simboli.apply(lambda x: Detector(x, quiet=True))
df_test['text_lang'] = df_test.poly_obj.apply(lambda x: icu.Locale.getDisplayName(x.language.locale))
df_test['text_lan_code'] = df_test.poly_obj.apply( lambda x: x.language.code)
df_test['lang_confidence'] = df_test.poly_obj.apply( lambda x: x.language.confidence)

In [32]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 70772 to 22816
Data columns (total 6 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   text_nohash_men_http_nl_2spazi_num_simboli  3 non-null      object 
 1   text                                        3 non-null      object 
 2   poly_obj                                    3 non-null      object 
 3   text_lang                                   3 non-null      object 
 4   text_lan_code                               3 non-null      object 
 5   lang_confidence                             3 non-null      float64
dtypes: float64(1), object(5)
memory usage: 168.0+ bytes


In [33]:
df_test

,text_nohash_men_http_nl_2spazi_num_simboli,text,poly_obj,text_lang,text_lan_code,lang_confidence
70772,These are the matches you want to play in Djokovic v Zverev It s a blockbuster in Turin,These are the matches you want to play in.\n\n🇷🇸 Djokovic v. Zverev 🇩🇪\n\nIt's a blockbuster in Turin. #NittoATPFinals https://t.co/kCOMHO31Vk,Prediction is reliable: True\nLanguage 1: name: English code: en confidence: 98.0 read bytes: 1117\nLanguage 2: name: un code: un confidence: 0.0 read bytes: 0\nLanguage 3: name: un code: un confidence: 0.0 read bytes: 0,English,en,98.0
9482,From one to the other The only two Italians to win a singles match at the ATP Finals Jannik Sinner amp Matteo Berrettini,From one to the other 👊\n\nThe only two Italians to win a singles match at the ATP Finals: Jannik Sinner &amp; Matteo Berrettini\n\n@janniksin @MattBerrettini #NittoATPFinals https://t.co/7mFVhgiY4u,Prediction is reliable: True\nLanguage 1: name: English code: en confidence: 99.0 read bytes: 888\nLanguage 2: name: un code: un confidence: 0.0 read bytes: 0\nLanguage 3: name: un code: un confidence: 0.0 read bytes: 0,English,en,99.0
22816,Que c est cruel pour BerrettiniLe héros local doit abandonner en début de er set sur une blessure aux abdos alors qu il faisait presque jeu égal avec Alexander Zverev ab,Que c'est cruel pour Berrettini\n\nLe héros local doit abandonner en début de 1er set sur une blessure aux abdos alors qu'il faisait presque jeu égal avec Alexander Zverev (7-6 1-0 ab) #ATPFinals https://t.co/xrY3bYMxj0,Prediction is reliable: True\nLanguage 1: name: French code: fr confidence: 99.0 read bytes: 686\nLanguage 2: name: un code: un confidence: 0.0 read bytes: 0\nLanguage 3: name: un code: un confidence: 0.0 read bytes: 0,French,fr,99.0


In [34]:
df_test['poly_obj'] = df_test['poly_obj'].astype(str)
df_test_predict = df_test['poly_obj'].str.split(" ", expand=True)
df_test_predict.rename(columns={3:'prediction'}, inplace=True)
df_test_predict = df_test_predict.drop([0, 1, 2, 4, 5, 6], axis=1)
df_test_predict = df_test_predict.iloc[:,0:1]
df_test_predict['prediction_reliable'] = df_test_predict['prediction'].str.replace(r'\n([-A-Za-z0-9_.]+)', '') 
df_test_predict = df_test_predict.drop(['prediction'], axis=1)
df_test = pd.concat([df_test, df_test_predict], axis=1) 
df_test = df_test.drop(['poly_obj'], axis=1)
df_test

,text_nohash_men_http_nl_2spazi_num_simboli,text,text_lang,text_lan_code,lang_confidence,prediction_reliable
70772,These are the matches you want to play in Djokovic v Zverev It s a blockbuster in Turin,These are the matches you want to play in.\n\n🇷🇸 Djokovic v. Zverev 🇩🇪\n\nIt's a blockbuster in Turin. #NittoATPFinals https://t.co/kCOMHO31Vk,English,en,98.0,True
9482,From one to the other The only two Italians to win a singles match at the ATP Finals Jannik Sinner amp Matteo Berrettini,From one to the other 👊\n\nThe only two Italians to win a singles match at the ATP Finals: Jannik Sinner &amp; Matteo Berrettini\n\n@janniksin @MattBerrettini #NittoATPFinals https://t.co/7mFVhgiY4u,English,en,99.0,True
22816,Que c est cruel pour BerrettiniLe héros local doit abandonner en début de er set sur une blessure aux abdos alors qu il faisait presque jeu égal avec Alexander Zverev ab,Que c'est cruel pour Berrettini\n\nLe héros local doit abandonner en début de 1er set sur une blessure aux abdos alors qu'il faisait presque jeu égal avec Alexander Zverev (7-6 1-0 ab) #ATPFinals https://t.co/xrY3bYMxj0,French,fr,99.0,True


In [35]:
# apply polyglot
df['poly_obj'] = df['text_nohash_men_http_nl_2spazi_num_simboli'].apply(lambda x: Detector(x, quiet=True))
df['text_lang'] = df.poly_obj.apply(lambda x: icu.Locale.getDisplayName(x.language.locale))
df['text_lan_code'] = df.poly_obj.apply( lambda x: x.language.code)
df['lang_confidence'] = df.poly_obj.apply( lambda x: x.language.confidence)

Output streaming troncato alle ultime 5000 righe.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect t

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84912 entries, 0 to 84911
Data columns (total 23 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   id                                          84912 non-null  int64  
 1   user                                        84912 non-null  object 
 2   date                                        84912 non-null  object 
 3   text                                        84912 non-null  object 
 4   favorite_count                              84912 non-null  int64  
 5   hashtags                                    84912 non-null  object 
 6   location                                    50798 non-null  object 
 7   retweet                                     84912 non-null  bool   
 8   retweet_count                               84912 non-null  int64  
 9   followers_count                             84912 non-null  int64  
 10  in_reply_t

In [37]:
df['poly_obj'] = df['poly_obj'].astype(str)
df_det_lang = df['poly_obj'].str.split(" ", expand=True)
df_det_lang.rename(columns={3:'prediction'}, inplace=True)
df_det_lang = df_det_lang.drop([0, 1, 2, 4, 5, 6], axis=1)
df_det_lang = df_det_lang.iloc[:,0:1]
df_det_lang['prediction_reliable'] = df_det_lang['prediction'].str.replace(r'\n([-A-Za-z0-9_.]+)', '') 
df_det_lang = df_det_lang.drop(['prediction'], axis=1)
df = pd.concat([df, df_det_lang], axis=1) 
df = df.drop(['poly_obj'], axis=1)
df.head()

,id,user,date,text,favorite_count,hashtags,location,retweet,retweet_count,followers_count,in_reply_to_status_id,user_mentions,text_nohash,text_nohash_men_,text_nohash_men_http,text_nohash_men_http_nl,text_nohash_men_http_nl_2spazi,text_nohash_men_http_nl_2spazi_num,text_nohash_men_http_nl_2spazi_num_simboli,text_lang,text_lan_code,lang_confidence,prediction_reliable
0,1460367104297820165,mandt610,Mon Nov 15 22:00:24 +0000 2021,Rublev’s revenge 😤\n\n@AndreyRublev97 reverses the result from the 2020 #NittoATPFinals and gets the win over Tsitsipas! https://t.co/vQoi9wxPch,0,['NittoATPFinals'],NaN,False,100,671,NaN,"['atptour', 'AndreyRublev97']",Rublev’s revenge 😤\n\n@AndreyRublev97 reverses the result from the 2020 and gets the win over Tsitsipas! https://t.co/vQoi9wxPch,Rublev’s revenge 😤\n\nreverses the result from the 2020 and gets the win over Tsitsipas! https://t.co/vQoi9wxPch,Rublev’s revenge 😤\n\nreverses the result from the 2020 and gets the win over Tsitsipas!,Rublev’s revenge 😤reverses the result from the 2020 and gets the win over Tsitsipas! \n,Rublev’s revenge 😤reverses the result from the 2020 and gets the win over Tsitsipas!,Rublev’s revenge 😤reverses the result from the and gets the win over Tsitsipas!,Rublev s revenge reverses the result from the and gets the win over Tsitsipas,English,en,98.0,True
1,1460367139299282955,LeeFergusson,Mon Nov 15 22:00:32 +0000 2021,Red-hot start 🔥\n\n@AndreyRublev97 powers past 2019 champion Tsitsipas 6-4 6-4 to get his first win of the 2021 #NittoATPFinals https://t.co/mv9gTC9m0N,0,['NittoATPFinals'],"Gillingham, England",False,97,6998,NaN,"['TennisTV', 'AndreyRublev97']",Red-hot start 🔥\n\n@AndreyRublev97 powers past 2019 champion Tsitsipas 6-4 6-4 to get his first win of the 2021 https://t.co/mv9gTC9m0N,Red-hot start 🔥\n\npowers past 2019 champion Tsitsipas 6-4 6-4 to get his first win of the 2021 https://t.co/mv9gTC9m0N,Red-hot start 🔥\n\npowers past 2019 champion Tsitsipas 6-4 6-4 to get his first win of the 2021,Red-hot start 🔥powers past 2019 champion Tsitsipas 6-4 6-4 to get his first win of the 2021 \n,Red-hot start 🔥powers past 2019 champion Tsitsipas 6-4 6-4 to get his first win of the 2021,Red-hot start 🔥powers past champion Tsitsipas - - to get his first win of the,Red hot start powers past champion Tsitsipas to get his first win of the,English,en,98.0,True
2,1460367219548823556,73Satta,Mon Nov 15 22:00:52 +0000 2021,Alexander Zverev #NittoATPFinals https://t.co/NN9iNnheHA,0,['NittoATPFinals'],NaN,False,2,1117,NaN,['saschashoulders'],Alexander Zverev https://t.co/NN9iNnheHA,Alexander Zverev https://t.co/NN9iNnheHA,Alexander Zverev,Alexander Zverev \n,Alexander Zverev,Alexander Zverev,Alexander Zverev,Slovak,sk,94.0,False
3,1460367236573638665,bikson82,Mon Nov 15 22:00:56 +0000 2021,When the 🐺 met the 🦁\n\n@DjokerNole 🤝 @Ibra_official \n\n#NittoATPFinals https://t.co/za2aB5HnIJ,0,['NittoATPFinals'],"Zrenjanin, Srbija",False,749,999,NaN,"['atptour', 'DjokerNole', 'Ibra_official']",When the 🐺 met the 🦁\n\n@DjokerNole 🤝 @Ibra_official \n\nhttps://t.co/za2aB5HnIJ,When the 🐺 met the 🦁\n\n🤝 \n\nhttps://t.co/za2aB5HnIJ,When the 🐺 met the 🦁\n\n🤝 \n\n,When the 🐺 met the 🦁🤝 \n,When the 🐺 met the 🦁🤝,When the 🐺 met the 🦁🤝,When the met the,English,en,94.0,True
4,1460367240960917507,BarthelSamantha,Mon Nov 15 22:00:57 +0000 2021,💙🤍💙🤍🇬🇷 #Tsitsipas #NittoATPFinals https://t.co/8gFUvYyqiU,0,"['Tsitsipas', 'NittoATPFinals']",South of France 🇫🇷,False,0,157,NaN,[],💙🤍💙🤍🇬🇷 https://t.co/8gFUvYyqiU,💙🤍💙🤍🇬🇷 https://t.co/8gFUvYyqiU,💙🤍💙🤍🇬🇷,💙🤍💙🤍🇬🇷 \n,💙🤍💙🤍🇬🇷,💙🤍💙🤍🇬🇷,,un,un,0.0,False


In [38]:
df.shape

(84912, 23)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84912 entries, 0 to 84911
Data columns (total 23 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   id                                          84912 non-null  int64  
 1   user                                        84912 non-null  object 
 2   date                                        84912 non-null  object 
 3   text                                        84912 non-null  object 
 4   favorite_count                              84912 non-null  int64  
 5   hashtags                                    84912 non-null  object 
 6   location                                    50798 non-null  object 
 7   retweet                                     84912 non-null  bool   
 8   retweet_count                               84912 non-null  int64  
 9   followers_count                             84912 non-null  int64  
 10  in_reply_t

In [40]:
twe = df['id'].count()
lan = df['text_lang'].nunique()
print(f'{lan} lingue differenti per {twe} tweet di cui:')
df.groupby(['text_lang']).size().nlargest(10).reset_index(name='top10')

127 lingue differenti per 84912 tweet di cui:


,text_lang,top10
0,English,49233
1,Italian,15968
2,Spanish,5065
3,French,4570
4,un,1676
5,Portuguese,924
6,Polish,745
7,Japanese,583
8,German,459
9,Slovak,429


In [41]:
df.to_csv(dw_root_dir+'twitter_text_cleaned_lang_det.csv', index=False, sep=',', encoding='utf-8')

# 4. Player detection
ANTONELLO M.

In [ ]:
df = pd.read_csv(f'{st_man_root_dir}twitter_lang.csv')

In [ ]:
# List
Medvedev = ['Medvedev' or '#Medvedev' or '@DaniilMedwed']
Zverev = ['Zverev' or '#Zverev' '@AlexZverev']
Berrettini = ['Berrettini' or '#Berrettini' or '@MattBerrettini']
Hurkacz = ['Hurkacz' or '#Hurkacz' or '@HubertHurkacz']
Sinner = ['Sinner' or '#Sinner' or '@janniksin']
Djokovic = ['Djokovic' or '#Djokovic' or '@DjokerNole']
Tsitsipas = ['Tsitsipas' or '#Tsitsipas' or '@steftsitsipas']
Rublev = ['Rublev' or '#Rublev' or '@AndreyRublev97']
Ruud = ['Ruud' or '#Ruud' or '@CasperRuud98']
Norrie = ['Norrie' or '#Norrie' or '@cam_norrie']
ATP Finals = ['NittoATPFinals' or 'TennisTakesOverTorino' or '#ATPFinals' or 'ATPFinals' or 'ATP Finals']

# Player
Medvedev = 'Medvedev'
Zverev = 'Zverev'
Berrettini = 'Berrettini'
Hurkacz = 'Hurkacz'
Sinner = 'Sinner'
Djokovic = 'Djokovic'
Tsitsipas = 'Tsitsipas'
Rublev = 'Rublev'
Ruud = 'Ruud'
Norrie = 'Norrie'
ATP Finals = 'ATP Finals'

# Match
Djokovic and Ruud = 'Djokovic VS Ruud RR' # RR is Round Robin cioè fase a gironi
Rublev and Tsitsipas = 'Rublev VS Tsitsipas RR'
Djokovic and Rublev = 'Djokovic VS Rublev RR'
Ruud and Norrie = 'Ruud VS Norrie RR'
Ruud and Rublev = 'Ruud VS Rublev RR'
Djokovic and Norrie = 'Djokovic VS Norrie RR'
Medvedev and Hurkacz = 'Medvedev VS Hurkacz RR'
Zverev and Berrettini = 'Zverev VS Berrettini RR'
Medvedev and Zverev = 'Medvedev VS Zverev RR'
Sinner and Hurkacz = 'Sinner VS Hurkacz RR'
Zverev and Hurkacz = 'Zverev VS Hurkacz RR'
Medvedev and Sinner = 'Medvedev VS Sinner RR'
Djokovic and Zverev = 'Djokovic VS Zverev SF' # SF is Semi-Final
Medvedev and Ruud = 'Medvedev VS Ruud SF'
Medvedev and Zverev = 'Medvedev VS Zverev F' #F is Final

In [ ]:
data = {'Medvedev': [0, 0, 1, 0], 'Zverev': [0, 1, 1, 0], 'Berrettini': [0, 0, 1, 1], 'Hurkacz': [1, 0, 0, 0],  'ATP Finals': [1, 0, 0, 0]}
data = pd.DataFrame(data)
data

,Medvedev,Zverev,Berrettini,Hurkacz,ATP Finals
0,0,0,0,1,1
1,0,1,0,0,0
2,1,1,1,0,0
3,0,0,1,0,0


1. Si creano tante colonne quante sono le variabili in list (cella iniziale)
2. Se una delle parole definite nella lista (.str.contains) è presente si mette 1 colonna corrispondente corrispondete.
3. Se la somma delle nuove colonne è 0 il topic è Other
4. Se la somma delle nuove colonne è 1 il topic è il nome della colonna con valore 1.
5. Se la somma delle nuove colonne è 2 e ATP Finals = 1 allora il topic è il nome dell'altra colonna
6. Se la somma delle nuove colonne è 2 e ATP Finals = 0 allora il topic è il nome previsto da match se le colonne corrispettive sono a 1, altrimenti il topic è il nome di entrambi i giocatori.
7. Se la somma delle nuove colonne è 3 e ATP Finals = 1 allora il topic è il nome previsto da match se le colonne corrispettive sono a 1, altrimenti il topic è il nome di tutti i giocatori = 1.
8. Se la somma delle nuove colonne è 3 e ATP Finals = 0 il topic sono tutti i nomi dei giocatori = 1.
9. Se la somma delle nuove colonne è > 3 il topic è il nome di tutti i giocatori = 1

# 5. Sentiment
MARCO

In [ ]:
pip install vader-multi

     |████████████████████████████████| 126 kB 5.2 MB/s 


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/nucleare/DWH/twitter_text_cleaned.csv')
df.head()

,Unnamed: 0,id,user,date,text,favorite_count,hashtags,location,retweet,retweet_count,followers_count,in_reply_to_status_id,user_mentions,text_cleaned
0,0,1460367104297820165,mandt610,Mon Nov 15 22:00:24 +0000 2021,Rublev’s revenge 😤\n\n@AndreyRublev97 reverses...,0,['NittoATPFinals'],NaN,False,100,671,NaN,"['atptour', 'AndreyRublev97']",Rublev’s revenge 😤reverses the result from the...
1,1,1460367139299282955,LeeFergusson,Mon Nov 15 22:00:32 +0000 2021,Red-hot start 🔥\n\n@AndreyRublev97 powers past...,0,['NittoATPFinals'],"Gillingham, England",False,97,6998,NaN,"['TennisTV', 'AndreyRublev97']",Red-hot start 🔥powers past champion Tsitsipas...
2,2,1460367219548823556,73Satta,Mon Nov 15 22:00:52 +0000 2021,Alexander Zverev #NittoATPFinals https://t.co/...,0,['NittoATPFinals'],NaN,False,2,1117,NaN,['saschashoulders'],Alexander Zverev
3,3,1460367236573638665,bikson82,Mon Nov 15 22:00:56 +0000 2021,When the 🐺 met the 🦁\n\n@DjokerNole 🤝 @Ibra_of...,0,['NittoATPFinals'],"Zrenjanin, Srbija",False,749,999,NaN,"['atptour', 'DjokerNole', 'Ibra_official']",When the 🐺 met the 🦁🤝
4,4,1460367240960917507,BarthelSamantha,Mon Nov 15 22:00:57 +0000 2021,💙🤍💙🤍🇬🇷 #Tsitsipas #NittoATPFinals https://t.co...,0,"['Tsitsipas', 'NittoATPFinals']",South of France 🇫🇷,False,0,157,NaN,[],💙🤍💙🤍🇬🇷


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


vader=[]

analyzer = SentimentIntensityAnalyzer()

for x in (df.loc[0:150,'text_cleaned']):
  a = analyzer.polarity_scores(x)
  vader.append(a)

In [ ]:
df1 = pd.DataFrame(vader)
df1.shape

(151, 4)

In [ ]:
print(df1)
df1.to_csv('/content/drive/MyDrive/nucleare/SENTIMENT/prova1.csv')

       neg    neu    pos  compound
0    0.151  0.667  0.182    0.1759
1    0.000  0.628  0.372    0.8271
2    0.000  1.000  0.000    0.0000
3    0.000  1.000  0.000    0.0000
4    0.000  1.000  0.000    0.0000
..     ...    ...    ...       ...
146  0.000  1.000  0.000    0.0000
147  0.000  0.745  0.255    0.7579
148  0.000  1.000  0.000    0.0000
149  0.000  1.000  0.000    0.0000
150  0.151  0.667  0.182    0.1759

[151 rows x 4 columns]


# 6. BoW



In [ ]:
import warnings
import nltk ### libreria di linguistica (Stanford)
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer #### individua la radice linguistica delle parole: non tronca le parole come un stamming
nltk.download('wordnet') ## tassonomia lingua inglese
nltk.download('stopwords')
from pprint import pprint
from gensim.models.phrases import Phrases
from nltk.corpus import stopwords
connectors = stopwords.words('english')

from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
file_path = '/content/drive/MyDrive/nucleare/ETL/twitter_merged_from_scraping.csv'

df = pd.read_csv(file_path)

lemmatizer = WordNetLemmatizer()
warnings.filterwarnings('ignore')

text = df.text
print(len(text))
text = text[~(text.isna())]
####pulizia: sostituisco tutti i caratteri speciali con uno spazio
text = text.str.lower().str.replace('[^\w\s\d]',' ', regex=True) 
text = text.str.replace('\n','', regex=False) 
text = text.str.replace('\s+', ' ', regex=True) ## rimuovi spazi multipli
print(len(text))

###TOKENIZATION
token = text.str.split()

###LEMMATIZATION: recupero radice parole
ltoken = token.apply(lambda x: [lemmatizer.lemmatize(sent) for sent in x])

### STOP WORDS REMOVING
stop.extend(['good', 'bad', 'dont', 'many', 'love', 'excellent', 'would', 'perfect', 'even', 'great','nice', 'amazing'])
train_sentences = []
for row in ltoken:
    train_sentences.append([item for item in row if item not in stop])

ltoken = train_sentences

print(f'token[0]: {token[0]}')
print(f'ltoken[0]: {ltoken[0]}')

### BIGRAM
bigram = Phrases(ltoken, min_count=5, threshold=0.2, common_terms=connectors) #gensim 3, ad esempio "and" 
bigrams = [bigram[item] for item in ltoken]

### NGRAMS
ngram = Phrases(bigrams, min_count=5, threshold=0.2, common_terms=connectors) # gensim 3
ngrams = [ngram[item] for item in bigrams]

print(f'bigrams[0]: {bigrams[0]}')
print(f'ngrams[0]: {ngrams[0]}')

linearized_ngram_texts_list = [' '.join(ngrams_item) for ngrams_item in ngrams]
print(linearized_ngram_texts_list[1])

linearized_bgram_texts_list = [' '.join(bgrams_item) for bgrams_item in bigrams]
print(linearized_bgram_texts_list[1])

33480
33480
token[0]: ['rublev', 's', 'revenge', 'andreyrublev97', 'reverses', 'the', 'result', 'from', 'the', '2020', 'nittoatpfinals', 'and', 'gets', 'the', 'win', 'over', 'tsitsipas', 'https', 't', 'co', 'vqoi9wxpch']
ltoken[0]: ['rublev', 'revenge', 'andreyrublev97', 'revers', 'result', '2020', 'nittoatpfinals', 'get', 'win', 'tsitsipas', 'http', 'co', 'vqoi9wxpch']
bigrams[0]: ['rublev_revenge', 'andreyrublev97_revers', 'result_2020', 'nittoatpfinals_get', 'win_tsitsipas', 'http_co', 'vqoi9wxpch']
ngrams[0]: ['rublev_revenge_andreyrublev97_revers', 'result_2020_nittoatpfinals_get', 'win_tsitsipas_http_co', 'vqoi9wxpch']


In [ ]:
saving_path = '/content/drive/MyDrive/nucleare/tone/'
df_n  = pd.DataFrame(linearized_ngram_texts_list, columns=['ngrams'])
df_bi = pd.DataFrame(linearized_bgram_texts_list, columns=['bigrams'])

df_bi.to_csv(saving_path + 'bigrams.csv', index=False)
df_n.to_csv(saving_path + 'ngrams.csv', index=False)

df_bi.head()

####################################################
####################################################
 #### TO BE CONTINUED
####################################################
####################################################

,bigrams
0,rublev_revenge andreyrublev97_revers result_20...
1,red_hot start_andreyrublev97 power_past 2019_c...
2,alexander_zverev nittoatpfinals_http co nn9inn...
3,met_djokernole ibra_official_nittoatpfinals ht...
4,tsitsipas_nittoatpfinals http_co 8gfuvyyqiu


# 7. Word embeddings
TONE'

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
33480
33480


0        rublev s revenge andreyrublev97 reverses the r...
1        red hot start andreyrublev97 powers past 2019 ...
2        alexander zverev nittoatpfinals https t co nn9...
3        when the met the djokernole ibra_official nitt...
4           tsitsipas nittoatpfinals https t co 8gfuvyyqiu
                               ...                        
33475    alexander zverev vs hubert hurkacz odds amp pr...
33476     atpfinals zverev 2 0 1 67 1 medvedev 2 0 1 95...
33477    vigrin atp afraid to address zverev s casechad...
33478     sg_lockwood bah ecoute chacun son delire depu...
33479     theborisbecker hi boris will you comment tomo...
Name: text, Length: 33480, dtype: object

# 8. Lime

# 9. SocialNetwork Analisys